In [1]:
!ls data/han/AWS/

남이섬_2016.xlsx  남이섬_2020.xlsx  청평_2019.xlsx  화천_2018.xlsx
남이섬_2017.xlsx  청평_2016.xlsx    청평_2020.xlsx  화천_2019.xlsx
남이섬_2018.xlsx  청평_2017.xlsx    화천_2016.xlsx  화천_2020.xlsx
남이섬_2019.xlsx  청평_2018.xlsx    화천_2017.xlsx


In [2]:
!pwd

/home/cjinw/work/water-quality/rnn


In [3]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [6]:

# folder = 'data/han/자동'
# folder = 'data/han/수질'
# folder = 'data/han/AWS'
folder = 'data/nakdong/자동'

# ## 한강
# ## 자동
# file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
#               ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
#               ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
# file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
#               ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
#               ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]
 

    
## 낙동강
#자동
file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
              ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
              ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]


## 금강
#자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]
             

    
    
## 영산강
#자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]
    


In [7]:
target_col = '용존산소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,2:11', ':,28:45', ':,26:29']
nakdong = [':,2:11', ':,2:12']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False
# gain_save_path = gain_han[2]

iloc_val = han[0]
iloc_val

':,2:11'

In [8]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [9]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [10]:
def normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_mean = df_all.mean()
    train_std = df_all.std()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/train_std
    
    
    return df_all, train_mean, train_std

In [11]:
def min_max_normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_min = df_all.min()
    train_max = df_all.max()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_min)/(train_max - train_min)       
    
    
    return df_all, train_min, train_max

In [12]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df = make_dataframe(file_names, iloc_val, interpolate=interpolation_option)
df_all, train_mean, train_std = normalize(df)

도개_2016.xlsx
(8784, 12)
(8784, 12)
도개_2017.xlsx
(8760, 12)
(8760, 12)
도개_2018.xlsx
(8755, 12)
(8760, 12)
도개_2019.xlsx
(8760, 12)
(8760, 12)
신암_2016.xlsx
(8742, 11)
(8784, 11)
신암_2017.xlsx
(8744, 11)
(8760, 11)
신암_2018.xlsx
(8758, 11)
(8760, 11)
신암_2019.xlsx
(8759, 11)
(8760, 11)
회상_2016.xlsx
(8784, 27)
(8784, 27)
회상_2017.xlsx
(8742, 27)
(8760, 27)
회상_2018.xlsx
(8760, 27)
(8760, 27)
회상_2019.xlsx
(8760, 27)
(8760, 27)


In [13]:
train_mean, train_std

(수온          1.620074e+01
 수소이온농도      7.959209e+00
 전기전도도       2.361947e+02
 용존산소        1.029059e+01
 탁도          1.139964e+01
 총유기탄소       2.949737e+00
 총질소         2.511964e+00
 총인          1.931998e-02
 클로로필-a      1.822085e+01
 Day sin     6.037103e-15
 Day cos     2.747682e-15
 Year sin   -5.681266e-08
 Year cos    2.053380e-05
 dtype: float64,
 수온           9.217941
 수소이온농도       0.600598
 전기전도도       44.257904
 용존산소         2.455950
 탁도          25.770777
 총유기탄소        0.885916
 총질소          0.791127
 총인           0.022864
 클로로필-a      14.528678
 Day sin      0.707110
 Day cos      0.707110
 Year sin     0.707103
 Year cos     0.707117
 dtype: float64)

In [14]:
df[2]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.377829,0.567419,1.690214,1.306788,-0.384142,-0.620530,0.437396,-0.407618,-0.586485,-1.999586e-12,1.414207,-0.003771,1.414158
1,-1.377829,0.567419,1.690214,1.306788,-0.384142,-0.620530,0.479109,-0.495090,-0.572719,3.660237e-01,1.366019,-0.002757,1.414161
2,-1.388677,0.567419,1.712809,1.306788,-0.388023,-0.620530,0.500597,-0.363882,-0.586485,7.071034e-01,1.224739,-0.001743,1.414162
3,-1.377829,0.567419,1.712809,1.266071,-0.384142,-0.620530,0.475317,-0.451354,-0.565836,9.999952e-01,0.999995,-0.000730,1.414163
4,-1.388677,0.567419,1.712809,1.225353,-0.384142,-0.620530,0.494277,-0.538826,-0.565836,1.224739e+00,0.707103,0.000284,1.414163
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.334435,-1.597091,0.085981,-0.851236,-0.364740,0.621123,-1.261445,0.335893,-1.199066,-1.366019e+00,0.366024,-0.008109,1.414140
35060,-1.334435,-1.597091,0.085981,-0.851236,-0.364740,0.621123,-1.270293,0.335893,-1.199066,-1.224739e+00,0.707103,-0.007096,1.414145
35061,-1.334435,-1.597091,0.085981,-0.851236,-0.364740,0.621123,-1.282934,0.292157,-1.199066,-9.999952e-01,0.999995,-0.006082,1.414150
35062,-1.345284,-1.597091,0.085981,-0.851236,-0.364740,0.621123,-1.263973,0.248421,-1.199066,-7.071034e-01,1.224739,-0.005068,1.414154


In [15]:
df[2]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.377829,0.567419,1.690214,1.306788,-0.384142,-0.620530,0.437396,-0.407618,-0.586485,-1.999586e-12,1.414207,-0.003771,1.414158
1,-1.377829,0.567419,1.690214,1.306788,-0.384142,-0.620530,0.479109,-0.495090,-0.572719,3.660237e-01,1.366019,-0.002757,1.414161
2,-1.388677,0.567419,1.712809,1.306788,-0.388023,-0.620530,0.500597,-0.363882,-0.586485,7.071034e-01,1.224739,-0.001743,1.414162
3,-1.377829,0.567419,1.712809,1.266071,-0.384142,-0.620530,0.475317,-0.451354,-0.565836,9.999952e-01,0.999995,-0.000730,1.414163
4,-1.388677,0.567419,1.712809,1.225353,-0.384142,-0.620530,0.494277,-0.538826,-0.565836,1.224739e+00,0.707103,0.000284,1.414163
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.334435,-1.597091,0.085981,-0.851236,-0.364740,0.621123,-1.261445,0.335893,-1.199066,-1.366019e+00,0.366024,-0.008109,1.414140
35060,-1.334435,-1.597091,0.085981,-0.851236,-0.364740,0.621123,-1.270293,0.335893,-1.199066,-1.224739e+00,0.707103,-0.007096,1.414145
35061,-1.334435,-1.597091,0.085981,-0.851236,-0.364740,0.621123,-1.282934,0.292157,-1.199066,-9.999952e-01,0.999995,-0.006082,1.414150
35062,-1.345284,-1.597091,0.085981,-0.851236,-0.364740,0.621123,-1.263973,0.248421,-1.199066,-7.071034e-01,1.224739,-0.005068,1.414154


In [16]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [17]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [18]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [19]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [20]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result
@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.val))
    # And cache it for next time
    self._example = result
  return result
@property
def example3(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2
WindowGenerator.example3 = example3

In [21]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [22]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [23]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [24]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [25]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [26]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [27]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [28]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [29]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[    0     9     2     0]
 [    2     7     1     9]
 [    3     1     1    16]
 ...
 [ 6936    14     3 20707]
 [ 6939    21     4 20721]
 [ 6943     1     1 20742]]
miss_data file saved


In [30]:
dgen = GainDataGenerator(df)

pattern none


In [31]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,4.7,7.8,333.0,11.1,22.5,1.4,NaN,0.004,6.7,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.7,7.8,333.0,11.1,22.5,1.3,NaN,0.007,6.8,2.588190e-01,0.965926,-0.001950,0.999998
2,4.7,7.8,333.0,11.0,22.6,1.3,NaN,0.007,7.0,5.000000e-01,0.866025,-0.001233,0.999999
3,4.7,7.8,333.0,11.0,22.5,1.3,NaN,0.005,7.0,7.071068e-01,0.707107,-0.000516,1.000000
4,4.7,7.8,333.0,11.0,22.5,1.4,NaN,0.005,7.1,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3.9,7.0,240.0,8.2,2.0,3.5,1.514,0.027,0.8,-9.659258e-01,0.258819,-0.005734,0.999984
35060,3.9,7.0,240.0,8.2,2.0,3.5,1.507,0.027,0.8,-8.660254e-01,0.500000,-0.005017,0.999987
35061,3.9,7.0,240.0,8.2,2.0,3.5,1.497,0.026,0.8,-7.071068e-01,0.707107,-0.004301,0.999991
35062,3.8,7.0,240.0,8.2,2.0,3.5,1.512,0.025,0.8,-5.000000e-01,0.866025,-0.003584,0.999994


In [32]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

## Graph 

In [33]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 13])

In [34]:
wide_window.plot(plot_col=target_col)
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  14384


In [35]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [36]:
val_performance = {}
performance = {}

In [37]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [38]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [39]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 111.0077 - disc_loss: 0.7211 - rmse: 1.0057 - val_loss: 0.9657
Epoch 2/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 97.3750 - disc_loss: 0.5169 - rmse: 0.9627 - val_loss: 0.8142
Epoch 3/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 69.3750 - disc_loss: 0.5499 - rmse: 0.7604 - val_loss: 0.7501
Epoch 4/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 63.3859 - disc_loss: 0.4024 - rmse: 0.7352 - val_loss: 0.6828
Epoch 5/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 54.9724 - disc_loss: 0.4147 - rmse: 0.7145 - val_loss: 0.6759
Epoch 6/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 48.2248 - disc_loss: 0.4111 - rmse: 0.6587 - val_loss: 0.6843
Epoch 7/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 42.3925 - disc_loss: 0.4041 - rmse: 0.6173 - val_loss: 0.5909
Ep

1/1 [==============================] - 0s 157ms/step - gen_loss: 4.6100 - disc_loss: 0.2950 - rmse: 0.2510 - val_loss: 0.2887
Epoch 60/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 5.5618 - disc_loss: 0.2794 - rmse: 0.2841 - val_loss: 0.3136
Epoch 61/2000
1/1 [==============================] - 0s 146ms/step - gen_loss: 5.7993 - disc_loss: 0.2940 - rmse: 0.2807 - val_loss: 0.2804
Epoch 62/2000
1/1 [==============================] - 0s 104ms/step - gen_loss: 5.9129 - disc_loss: 0.2907 - rmse: 0.3363 - val_loss: 0.2865
Epoch 63/2000
1/1 [==============================] - 0s 136ms/step - gen_loss: 5.0962 - disc_loss: 0.2807 - rmse: 0.2663 - val_loss: 0.3349
Epoch 64/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 6.1677 - disc_loss: 0.2749 - rmse: 0.2993 - val_loss: 0.2905
Epoch 65/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 5.0472 - disc_loss: 0.2827 - rmse: 0.2872 - val_loss: 0.2755
Epoch 66/2000
1/1 [===============

1/1 [==============================] - 0s 128ms/step - gen_loss: 3.6663 - disc_loss: 0.2465 - rmse: 0.2471 - val_loss: 0.2449
Epoch 118/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 3.8210 - disc_loss: 0.2424 - rmse: 0.2418 - val_loss: 0.2494
Epoch 119/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 3.8501 - disc_loss: 0.2441 - rmse: 0.2500 - val_loss: 0.2850
Epoch 120/2000
1/1 [==============================] - 0s 125ms/step - gen_loss: 3.5483 - disc_loss: 0.2373 - rmse: 0.2437 - val_loss: 0.2738
Epoch 121/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 3.3494 - disc_loss: 0.2407 - rmse: 0.2328 - val_loss: 0.2304
Epoch 122/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 4.1650 - disc_loss: 0.2384 - rmse: 0.2406 - val_loss: 0.2513
Epoch 123/2000
1/1 [==============================] - 0s 148ms/step - gen_loss: 3.5385 - disc_loss: 0.2367 - rmse: 0.2453 - val_loss: 0.2402
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 158ms/step - gen_loss: 2.8931 - disc_loss: 0.2202 - rmse: 0.2054 - val_loss: 0.2418
Epoch 176/2000
1/1 [==============================] - 0s 148ms/step - gen_loss: 3.3192 - disc_loss: 0.2285 - rmse: 0.2182 - val_loss: 0.2144
Epoch 177/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 3.2426 - disc_loss: 0.2261 - rmse: 0.2195 - val_loss: 0.2425
Epoch 178/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 3.4788 - disc_loss: 0.2208 - rmse: 0.2323 - val_loss: 0.2313
Epoch 179/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 3.4334 - disc_loss: 0.2174 - rmse: 0.2026 - val_loss: 0.2426
Epoch 180/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 2.9474 - disc_loss: 0.2166 - rmse: 0.2264 - val_loss: 0.2076
Epoch 181/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 3.3126 - disc_loss: 0.2235 - rmse: 0.2369 - val_loss: 0.2375
Epoch 182/2000
1/1 [========

1/1 [==============================] - 0s 140ms/step - gen_loss: 2.8281 - disc_loss: 0.2100 - rmse: 0.2234 - val_loss: 0.2320
Epoch 234/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 3.2075 - disc_loss: 0.2027 - rmse: 0.2289 - val_loss: 0.2133
Epoch 235/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 3.2328 - disc_loss: 0.2046 - rmse: 0.2508 - val_loss: 0.2356
Epoch 236/2000
1/1 [==============================] - 0s 98ms/step - gen_loss: 2.5520 - disc_loss: 0.1995 - rmse: 0.2085 - val_loss: 0.2344
Epoch 237/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 2.7997 - disc_loss: 0.2027 - rmse: 0.2298 - val_loss: 0.2568
Epoch 238/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 3.4810 - disc_loss: 0.2017 - rmse: 0.2261 - val_loss: 0.2529
Epoch 239/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 2.5163 - disc_loss: 0.2001 - rmse: 0.2212 - val_loss: 0.2076
Epoch 240/2000
1/1 [=========

...

Epoch 1160/2000
1/1 [==============================] - 0s 175ms/step - gen_loss: 1.6995 - disc_loss: 0.1407 - rmse: 0.1574 - val_loss: 0.1972
Epoch 1161/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 1.7865 - disc_loss: 0.1356 - rmse: 0.1999 - val_loss: 0.1948
Epoch 1162/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 2.2126 - disc_loss: 0.1341 - rmse: 0.1831 - val_loss: 0.2186
Epoch 1163/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 1.7729 - disc_loss: 0.1389 - rmse: 0.2126 - val_loss: 0.3029
Epoch 1164/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 1.7627 - disc_loss: 0.1335 - rmse: 0.1789 - val_loss: 0.1965
Epoch 1165/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 1.9699 - disc_loss: 0.1396 - rmse: 0.1997 - val_loss: 0.2069
Epoch 1166/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 1.7534 - disc_loss: 0.1370 - rmse: 0.1793 - val_loss: 0.3007
Epoch 

1/1 [==============================] - 0s 124ms/step - gen_loss: 1.7543 - disc_loss: 0.1395 - rmse: 0.1656 - val_loss: 0.3654
Epoch 1218/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 1.7601 - disc_loss: 0.1319 - rmse: 0.1931 - val_loss: 0.1876
Epoch 1219/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 1.6718 - disc_loss: 0.1326 - rmse: 0.1869 - val_loss: 0.1905
Epoch 1220/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 1.7055 - disc_loss: 0.1341 - rmse: 0.1661 - val_loss: 0.2021
Epoch 1221/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 1.6410 - disc_loss: 0.1331 - rmse: 0.1524 - val_loss: 0.1869
Epoch 1222/2000
1/1 [==============================] - 0s 157ms/step - gen_loss: 1.8770 - disc_loss: 0.1293 - rmse: 0.1817 - val_loss: 0.3104
Epoch 1223/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 1.6816 - disc_loss: 0.1296 - rmse: 0.1772 - val_loss: 0.2000
Epoch 1224/2000
1/1 [=

1/1 [==============================] - 0s 142ms/step - gen_loss: 1.8276 - disc_loss: 0.1294 - rmse: 0.1761 - val_loss: 0.1900
Epoch 1275/2000
1/1 [==============================] - 0s 104ms/step - gen_loss: 1.8568 - disc_loss: 0.1288 - rmse: 0.1856 - val_loss: 0.1740
Epoch 1276/2000
1/1 [==============================] - 0s 163ms/step - gen_loss: 1.7412 - disc_loss: 0.1277 - rmse: 0.1663 - val_loss: 0.1899
Epoch 1277/2000
1/1 [==============================] - 0s 147ms/step - gen_loss: 1.6908 - disc_loss: 0.1337 - rmse: 0.1637 - val_loss: 0.1929
Epoch 1278/2000
1/1 [==============================] - 0s 145ms/step - gen_loss: 2.0348 - disc_loss: 0.1272 - rmse: 0.1880 - val_loss: 0.2063
Epoch 1279/2000
1/1 [==============================] - 0s 148ms/step - gen_loss: 1.6428 - disc_loss: 0.1284 - rmse: 0.1612 - val_loss: 0.1976
Epoch 1280/2000
1/1 [==============================] - 0s 147ms/step - gen_loss: 1.7683 - disc_loss: 0.1245 - rmse: 0.1729 - val_loss: 0.1787
Epoch 1281/2000
1/1 [=

1/1 [==============================] - 0s 124ms/step - gen_loss: 1.8792 - disc_loss: 0.1226 - rmse: 0.1787 - val_loss: 0.2098
Epoch 1332/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 1.6099 - disc_loss: 0.1241 - rmse: 0.1555 - val_loss: 0.1886
Epoch 1333/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 1.9577 - disc_loss: 0.1239 - rmse: 0.1850 - val_loss: 0.2058
Epoch 1334/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 1.8025 - disc_loss: 0.1246 - rmse: 0.1875 - val_loss: 0.1770
Epoch 1335/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 1.7757 - disc_loss: 0.1251 - rmse: 0.1579 - val_loss: 0.2025
Epoch 1336/2000
1/1 [==============================] - 0s 147ms/step - gen_loss: 1.8092 - disc_loss: 0.1240 - rmse: 0.1878 - val_loss: 0.2023
Epoch 1337/2000
1/1 [==============================] - 0s 147ms/step - gen_loss: 1.8230 - disc_loss: 0.1250 - rmse: 0.1594 - val_loss: 0.1816
Epoch 1338/2000
1/1 [=

1/1 [==============================] - 0s 126ms/step - gen_loss: 1.7927 - disc_loss: 0.1275 - rmse: 0.2031 - val_loss: 0.1717
load save
1/1 [==============================] - 0s 70ms/step - loss: 0.1983
load save


**학습 loss history 출력**

In [40]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [41]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 3s 28ms/step - loss: 0.2234


0.2233576774597168

**model save**

In [42]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [43]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
norm_df = pd.concat(df,axis=0)

In [45]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

105192
(120, 13)
120
x.shape = (105120, 13)
x_reshape.shape = (876, 120, 13)
108070
y_true.shape= (105120, 13)
108070


(1, 120, 13)

In [46]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

28/28 [==============================] - 0s 2ms/step - loss: 0.0000e+00


0.0

In [47]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [48]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(105120, 13) (120, 13)
(105192, 13)


In [49]:
y_pred[~np.isnan(data)] = np.nan

In [50]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Imputed Data**

In [51]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

105192
unit_shape= (120, 13)
120
n= 105120
(35040, 9)
(35040, 9)


# 예측

In [52]:
df[0].head

<bound method NDFrame.head of              수온    수소이온농도     전기전도도      용존산소        탁도     총유기탄소       총질소  \
0     -1.247648 -0.265085  2.187300  0.329570  0.430734 -1.749304       NaN   
1     -1.247648 -0.265085  2.187300  0.329570  0.430734 -1.862182       NaN   
2     -1.247648 -0.265085  2.187300  0.288852  0.434615 -1.862182       NaN   
3     -1.247648 -0.265085  2.187300  0.288852  0.430734 -1.862182       NaN   
4     -1.247648 -0.265085  2.187300  0.288852  0.430734 -1.749304       NaN   
...         ...       ...       ...       ...       ...       ...       ...   
35059 -1.345284  0.234418  0.696041 -0.077605 -0.322056 -0.846285  0.781209   
35060 -1.345284  0.234418  0.650852 -0.077605 -0.322056 -0.733407  0.860842   
35061 -1.345284  0.234418  0.605662 -0.118322 -0.325937 -0.733407  0.851994   
35062 -1.366980  0.234418  0.628257 -0.118322 -0.325937 -0.846285  0.844410   
35063 -1.366980  0.234418  0.628257 -0.118322 -0.325937 -0.846285  0.800169   

             총인    클로

In [53]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
print(oris[0].shape, gans[0].shape)
print(oris[1].shape, gans[1].shape)
print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

(35040, 13) (35040, 13)
(35040, 9) (35040, 9)
(35040, 31) (35040, 31)


In [54]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,-1.247648,-0.265085,2.187300,0.329570,0.430734,-1.749304,0.000345,-0.670034,-0.792973,-1.999586e-12,...,-1.040758,-1.377829,0.567419,1.690214,1.306788,-0.384142,-0.620530,0.437396,-0.407618,-0.586485
1,-1.247648,-0.265085,2.187300,0.329570,0.430734,-1.862182,-0.091246,-0.538826,-0.786090,3.660237e-01,...,-1.033876,-1.377829,0.567419,1.690214,1.306788,-0.384142,-0.620530,0.479109,-0.495090,-0.572719
2,-1.247648,-0.265085,2.187300,0.288852,0.434615,-1.862182,-0.100376,-0.538826,-0.772324,7.071034e-01,...,-1.033876,-1.388677,0.567419,1.712809,1.306788,-0.388023,-0.620530,0.500597,-0.363882,-0.586485
3,-1.247648,-0.265085,2.187300,0.288852,0.430734,-1.862182,-0.027109,-0.626298,-0.772324,9.999952e-01,...,-1.026993,-1.377829,0.567419,1.712809,1.266071,-0.384142,-0.620530,0.475317,-0.451354,-0.565836
4,-1.247648,-0.265085,2.187300,0.288852,0.430734,-1.749304,-0.117197,-0.626298,-0.765441,1.224739e+00,...,-1.013227,-1.388677,0.567419,1.712809,1.225353,-0.384142,-0.620530,0.494277,-0.538826,-0.565836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,-0.085025,-0.473421,-0.347409,-0.133167,-0.050414,0.395368,0.997356,0.729517,-0.579602,-1.366019e+00,...,-0.565836,0.010768,0.733920,-0.456295,0.981049,0.000014,0.395368,0.413380,0.248421,1.746832
24524,0.042455,-0.614758,-0.260280,-0.081889,0.099800,0.395368,1.127550,0.860725,-0.614016,-1.224739e+00,...,-0.586485,0.010768,0.733920,-0.456295,0.940331,0.015535,0.395368,0.342595,0.204685,1.753715
24525,-0.176383,-0.722987,-0.340537,-0.025070,0.038431,0.395368,1.047917,0.685781,-0.648431,-9.999952e-01,...,-0.497006,0.010768,0.733920,-0.433701,0.940331,0.038817,0.395368,0.313522,0.248421,1.788129
24526,-0.103215,-0.644509,-0.282555,-0.093308,-0.008522,0.395368,1.126286,0.816989,-0.655314,-7.071034e-01,...,-0.469475,-0.000081,0.567419,-0.433701,0.858896,0.054339,0.395368,0.297090,0.248421,1.643588


In [55]:
df[0].columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '탁도', '총유기탄소', '총질소', '총인', '클로로필-a',
       'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [56]:
MAX_EPOCHS = 150
target_col = '용존산소'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
out_features = [target_col_idx]
# out_features = [3,5,6,7,8]
out_num_features = len(out_features)
out_num_features

1

In [57]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [58]:
OUT_STEPS = 24*3

In [59]:
# OUT_STEPS = 24*3
# wdg = WaterDataGenerator(train_df,
#                          batch_size=128,
#                          input_width = 24*7,
#                          label_width = OUT_STEPS,
#                          shift = OUT_STEPS,
#                          out_num_features = out_num_features,
# #                          out_features = out_features
#                         )

In [60]:
# train_df

In [61]:
# it = iter(wdg)

In [62]:
# x,y = next(it)
# x.shape, y.shape

In [63]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [64]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [65]:
def hour_to_day_mean(array):
    
    time = 24
    return tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
    
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = (input_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = (label_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = (predict_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#       predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [66]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [67]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, windows=None, min_max_normailze=False):
       
    if example is not None :
        inputs, labels = windows
    else :
        inputs, labels = self.example
        
        
    plot_out_col_index = self.column_indices[plot_out_col]
    
    if self.label_columns:
        label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
        label_out_col_index = plot_out_col_index
    
    if model is None:
        return
    
    print("column : " + df_all.columns[plot_col])
    
    mae = 0
    mse = 0
    rmse = 0
    mape = 0
    pred_arr = []
    label_arr = []
    
    predictions = model(inputs)
    
    if min_max_normailze :
        predictions = predictions * train_std[plot_col] + train_mean[plot_col]
        labels = labels * train_std[plot_col] + train_mean[plot_col]
        
        predictions = predictions.numpy()
        labels = labels.numpy()
        
        predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())
        labels = (labels - labels.min()) / (labels.max() - labels.min())

    for n in range(len(inputs)):
        pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
        label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
        
        pred_arr.append(pred_temp[2])
        label_arr.append(label_temp[2])
        
        
    mae = mean_absolute_error(label_arr, pred_arr)
    mse = mean_squared_error(label_arr, pred_arr)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((np.array(label_arr) - np.array(pred_arr))/np.array(label_arr)) )*100
    
    print("mae:")  
    print(mae)
    
    print("mse:")  
    print(mse)
    
    print("rmse")  
    print(rmse)
    
    print("mape")  
    print(mape)
    
    

WindowGenerator.compa = compa

In [68]:
train_df.shape

(24528, 31)

In [69]:
#multi_window.compa(elman_model, plot_col=out_features[0])

In [70]:
# target_col = '용존산소'
# num_features = df[0].shape[1]
# label_columns_indices = {name: i for i, name in enumerate(df[0])}
# target_col_idx = label_columns_indices[target_col]
# # target_col_idx
# out_features = [target_col_idx]
# # out_features = [3,5,6,7,8]
# out_num_features = len(out_features)
# out_num_features

In [71]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )
multi_window

Total window size: 240
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167]
Label indices: [168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 

In [72]:
def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [73]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [74]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )

In [75]:
multi_val_performance = {}
multi_performance = {}

**Elman_RNN**

In [76]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [77]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [78]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/150
1/1 [==============================] - 1s 731ms/step - loss: 0.9842 - mean_absolute_error: 0.8274 - val_loss: 0.7622 - val_mean_absolute_error: 0.6391
Epoch 2/150
1/1 [==============================] - 0s 312ms/step - loss: 1.0033 - mean_absolute_error: 0.8230 - val_loss: 0.4370 - val_mean_absolute_error: 0.4925
Epoch 3/150
1/1 [==============================] - 0s 283ms/step - loss: 0.9223 - mean_absolute_error: 0.7936 - val_loss: 0.6669 - val_mean_absolute_error: 0.6076
Epoch 4/150
1/1 [==============================] - 0s 276ms/step - loss: 0.9034 - mean_absolute_error: 0.7839 - val_loss: 0.7440 - val_mean_absolute_error: 0.6251
Epoch 5/150
1/1 [==============================] - 0s 284ms/step - loss: 0.7558 - mean_absolute_error: 0.7202 - val_loss: 0.5188 - val_mean_absolute_error: 0.5125
Epoch 6/150
1/1 [==============================] - 0s 277ms/step - loss: 0.6661 - mean_absolute_error: 0.6662 - val_loss: 0.5195 - val_mean_absolute_error: 0.5041
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 318ms/step - loss: 0.1674 - mean_absolute_error: 0.3069 - val_loss: 0.2575 - val_mean_absolute_error: 0.3768
Epoch 52/150
1/1 [==============================] - 0s 242ms/step - loss: 0.1698 - mean_absolute_error: 0.2920 - val_loss: 0.1869 - val_mean_absolute_error: 0.3300
Epoch 53/150
1/1 [==============================] - 0s 267ms/step - loss: 0.2187 - mean_absolute_error: 0.3313 - val_loss: 0.2660 - val_mean_absolute_error: 0.3818
Epoch 54/150
1/1 [==============================] - 0s 285ms/step - loss: 0.1750 - mean_absolute_error: 0.3093 - val_loss: 0.2193 - val_mean_absolute_error: 0.3494
Epoch 55/150
1/1 [==============================] - 0s 278ms/step - loss: 0.1596 - mean_absolute_error: 0.2891 - val_loss: 0.1852 - val_mean_absolute_error: 0.3209
Epoch 56/150
1/1 [==============================] - 0s 266ms/step - loss: 0.1760 - mean_absolute_error: 0.3066 - val_loss: 0.1815 - val_mean_absolute_error: 0.3396
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 259ms/step - loss: 0.2023 - mean_absolute_error: 0.3104 - val_loss: 0.2111 - val_mean_absolute_error: 0.3509
Epoch 102/150
1/1 [==============================] - 0s 303ms/step - loss: 0.1500 - mean_absolute_error: 0.2648 - val_loss: 0.2224 - val_mean_absolute_error: 0.3672
Epoch 103/150
1/1 [==============================] - 0s 244ms/step - loss: 0.1498 - mean_absolute_error: 0.2630 - val_loss: 0.1932 - val_mean_absolute_error: 0.3293
Epoch 104/150
1/1 [==============================] - 0s 272ms/step - loss: 0.1203 - mean_absolute_error: 0.2513 - val_loss: 0.1609 - val_mean_absolute_error: 0.3134
Epoch 105/150
1/1 [==============================] - 0s 288ms/step - loss: 0.1246 - mean_absolute_error: 0.2542 - val_loss: 0.1827 - val_mean_absolute_error: 0.3340
Epoch 106/150
1/1 [==============================] - 0s 297ms/step - loss: 0.1443 - mean_absolute_error: 0.2588 - val_loss: 0.2668 - val_mean_absolute_error: 0.3854
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 85ms/step - loss: 0.1470 - mean_absolute_error: 0.2779
val performance = [0.20668458938598633, 0.3515920639038086]
test performance =  [0.14703187346458435, 0.2778961658477783]


In [79]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
multi_window.plot2(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [81]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [82]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/150
1/1 [==============================] - 1s 878ms/step - loss: 1.2759 - mean_absolute_error: 0.9451 - val_loss: 0.8712 - val_mean_absolute_error: 0.6774
Epoch 2/150
1/1 [==============================] - 0s 302ms/step - loss: 1.1706 - mean_absolute_error: 0.9087 - val_loss: 0.6655 - val_mean_absolute_error: 0.6093
Epoch 3/150
1/1 [==============================] - 0s 302ms/step - loss: 1.0483 - mean_absolute_error: 0.8697 - val_loss: 0.6915 - val_mean_absolute_error: 0.6229
Epoch 4/150
1/1 [==============================] - 0s 312ms/step - loss: 1.0289 - mean_absolute_error: 0.8231 - val_loss: 0.6296 - val_mean_absolute_error: 0.5555
Epoch 5/150
1/1 [==============================] - 0s 306ms/step - loss: 0.8014 - mean_absolute_error: 0.7316 - val_loss: 0.7316 - val_mean_absolute_error: 0.6314
Epoch 6/150
1/1 [==============================] - 0s 323ms/step - loss: 0.8172 - mean_absolute_error: 0.7492 - val_loss: 0.5759 - val_mean_absolute_error: 0.5344
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 336ms/step - loss: 0.1896 - mean_absolute_error: 0.3259 - val_loss: 0.3133 - val_mean_absolute_error: 0.4128
Epoch 52/150
1/1 [==============================] - 0s 302ms/step - loss: 0.1883 - mean_absolute_error: 0.3250 - val_loss: 0.2914 - val_mean_absolute_error: 0.4158
Epoch 53/150
1/1 [==============================] - 0s 308ms/step - loss: 0.2386 - mean_absolute_error: 0.3587 - val_loss: 0.3408 - val_mean_absolute_error: 0.4238
Epoch 54/150
1/1 [==============================] - 0s 306ms/step - loss: 0.2124 - mean_absolute_error: 0.3462 - val_loss: 0.3513 - val_mean_absolute_error: 0.4393
Epoch 55/150
1/1 [==============================] - 0s 309ms/step - loss: 0.1917 - mean_absolute_error: 0.3337 - val_loss: 0.2779 - val_mean_absolute_error: 0.4067
Epoch 56/150
1/1 [==============================] - 0s 307ms/step - loss: 0.2001 - mean_absolute_error: 0.3218 - val_loss: 0.3235 - val_mean_absolute_error: 0.4324
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 294ms/step - loss: 0.1444 - mean_absolute_error: 0.2722 - val_loss: 0.2161 - val_mean_absolute_error: 0.3527
Epoch 102/150
1/1 [==============================] - 0s 289ms/step - loss: 0.1724 - mean_absolute_error: 0.3070 - val_loss: 0.2510 - val_mean_absolute_error: 0.3692
Epoch 103/150
1/1 [==============================] - 0s 306ms/step - loss: 0.1505 - mean_absolute_error: 0.2823 - val_loss: 0.2226 - val_mean_absolute_error: 0.3691
Epoch 104/150
1/1 [==============================] - 0s 349ms/step - loss: 0.1457 - mean_absolute_error: 0.2654 - val_loss: 0.2967 - val_mean_absolute_error: 0.4136
Epoch 105/150
1/1 [==============================] - 0s 310ms/step - loss: 0.1385 - mean_absolute_error: 0.2612 - val_loss: 0.2512 - val_mean_absolute_error: 0.3982
Epoch 106/150
1/1 [==============================] - 0s 316ms/step - loss: 0.1516 - mean_absolute_error: 0.2863 - val_loss: 0.2477 - val_mean_absolute_error: 0.3861
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 94ms/step - loss: 0.1571 - mean_absolute_error: 0.2834
val performance = [0.20047318935394287, 0.33824512362480164]
test performance =  [0.1571260541677475, 0.28336721658706665]


**LSTM**

In [83]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [84]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/150
1/1 [==============================] - 2s 2s/step - loss: 1.1019 - mean_absolute_error: 0.8912 - val_loss: 0.6351 - val_mean_absolute_error: 0.5911
Epoch 2/150
1/1 [==============================] - 0s 258ms/step - loss: 1.0606 - mean_absolute_error: 0.8411 - val_loss: 0.7156 - val_mean_absolute_error: 0.5996
Epoch 3/150
1/1 [==============================] - 0s 274ms/step - loss: 1.0188 - mean_absolute_error: 0.8378 - val_loss: 0.5817 - val_mean_absolute_error: 0.5460
Epoch 4/150
1/1 [==============================] - 0s 269ms/step - loss: 1.1499 - mean_absolute_error: 0.9195 - val_loss: 0.5711 - val_mean_absolute_error: 0.5484
Epoch 5/150
1/1 [==============================] - 0s 259ms/step - loss: 0.9241 - mean_absolute_error: 0.8096 - val_loss: 0.5628 - val_mean_absolute_error: 0.5410
Epoch 6/150
1/1 [==============================] - 0s 270ms/step - loss: 0.8337 - mean_absolute_error: 0.7448 - val_loss: 0.3902 - val_mean_absolute_error: 0.4449
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 255ms/step - loss: 0.1764 - mean_absolute_error: 0.3164 - val_loss: 0.3190 - val_mean_absolute_error: 0.4110
Epoch 52/150
1/1 [==============================] - 0s 270ms/step - loss: 0.1728 - mean_absolute_error: 0.3048 - val_loss: 0.3443 - val_mean_absolute_error: 0.4406
Epoch 53/150
1/1 [==============================] - 0s 223ms/step - loss: 0.1611 - mean_absolute_error: 0.2941 - val_loss: 0.2419 - val_mean_absolute_error: 0.3716
Epoch 54/150
1/1 [==============================] - 0s 243ms/step - loss: 0.2102 - mean_absolute_error: 0.3281 - val_loss: 0.3106 - val_mean_absolute_error: 0.4192
Epoch 55/150
1/1 [==============================] - 0s 246ms/step - loss: 0.1412 - mean_absolute_error: 0.2704 - val_loss: 0.3157 - val_mean_absolute_error: 0.4250
Epoch 56/150
1/1 [==============================] - 0s 249ms/step - loss: 0.2274 - mean_absolute_error: 0.3403 - val_loss: 0.3121 - val_mean_absolute_error: 0.4247
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 262ms/step - loss: 0.1369 - mean_absolute_error: 0.2575 - val_loss: 0.2228 - val_mean_absolute_error: 0.3509
Epoch 102/150
1/1 [==============================] - 0s 272ms/step - loss: 0.1436 - mean_absolute_error: 0.2681 - val_loss: 0.2844 - val_mean_absolute_error: 0.3902
Epoch 103/150
1/1 [==============================] - 0s 246ms/step - loss: 0.1552 - mean_absolute_error: 0.2835 - val_loss: 0.2890 - val_mean_absolute_error: 0.4113
Epoch 104/150
1/1 [==============================] - 0s 266ms/step - loss: 0.1773 - mean_absolute_error: 0.2915 - val_loss: 0.2794 - val_mean_absolute_error: 0.4009
Epoch 105/150
1/1 [==============================] - 0s 246ms/step - loss: 0.1064 - mean_absolute_error: 0.2324 - val_loss: 0.3054 - val_mean_absolute_error: 0.4160
Epoch 106/150
1/1 [==============================] - 0s 243ms/step - loss: 0.1414 - mean_absolute_error: 0.2765 - val_loss: 0.2757 - val_mean_absolute_error: 0.4077
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 126ms/step - loss: 0.2213 - mean_absolute_error: 0.3493
val performance = [0.22128216922283173, 0.34932416677474976]
test performance =  [0.14757710695266724, 0.2851826846599579]


**CNN model**

In [85]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [86]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/150
1/1 [==============================] - 1s 769ms/step - loss: 1.2185 - mean_absolute_error: 0.9335 - val_loss: 0.8707 - val_mean_absolute_error: 0.6680
Epoch 2/150
1/1 [==============================] - 0s 268ms/step - loss: 0.9749 - mean_absolute_error: 0.8212 - val_loss: 0.6872 - val_mean_absolute_error: 0.5855
Epoch 3/150
1/1 [==============================] - 0s 255ms/step - loss: 0.9128 - mean_absolute_error: 0.7930 - val_loss: 0.6030 - val_mean_absolute_error: 0.5626
Epoch 4/150
1/1 [==============================] - 0s 244ms/step - loss: 0.6100 - mean_absolute_error: 0.6342 - val_loss: 0.4002 - val_mean_absolute_error: 0.4635
Epoch 5/150
1/1 [==============================] - 0s 255ms/step - loss: 0.3695 - mean_absolute_error: 0.4830 - val_loss: 0.3669 - val_mean_absolute_error: 0.4414
Epoch 6/150
1/1 [==============================] - 0s 240ms/step - loss: 0.3048 - mean_absolute_error: 0.4059 - val_loss: 0.3358 - val_mean_absolute_error: 0.4784
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 262ms/step - loss: 0.1689 - mean_absolute_error: 0.2842 - val_loss: 0.3626 - val_mean_absolute_error: 0.4475
Epoch 52/150
1/1 [==============================] - 0s 219ms/step - loss: 0.1684 - mean_absolute_error: 0.2953 - val_loss: 0.2265 - val_mean_absolute_error: 0.3586
Epoch 53/150
1/1 [==============================] - 0s 243ms/step - loss: 0.1608 - mean_absolute_error: 0.2865 - val_loss: 0.2743 - val_mean_absolute_error: 0.4064
Epoch 54/150
1/1 [==============================] - 0s 262ms/step - loss: 0.1582 - mean_absolute_error: 0.2794 - val_loss: 0.2419 - val_mean_absolute_error: 0.3733
Epoch 55/150
1/1 [==============================] - 0s 215ms/step - loss: 0.1507 - mean_absolute_error: 0.2715 - val_loss: 0.2745 - val_mean_absolute_error: 0.4108
Epoch 56/150
1/1 [==============================] - 0s 261ms/step - loss: 0.1670 - mean_absolute_error: 0.2821 - val_loss: 0.2249 - val_mean_absolute_error: 0.3720
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 256ms/step - loss: 0.1313 - mean_absolute_error: 0.2552 - val_loss: 0.2299 - val_mean_absolute_error: 0.3644
Epoch 102/150
1/1 [==============================] - 0s 256ms/step - loss: 0.0984 - mean_absolute_error: 0.2255 - val_loss: 0.1945 - val_mean_absolute_error: 0.3228
Epoch 103/150
1/1 [==============================] - 0s 230ms/step - loss: 0.1347 - mean_absolute_error: 0.2604 - val_loss: 0.1546 - val_mean_absolute_error: 0.3056
Epoch 104/150
1/1 [==============================] - 0s 241ms/step - loss: 0.1174 - mean_absolute_error: 0.2491 - val_loss: 0.2193 - val_mean_absolute_error: 0.3524
Epoch 105/150
1/1 [==============================] - 0s 248ms/step - loss: 0.1348 - mean_absolute_error: 0.2492 - val_loss: 0.1676 - val_mean_absolute_error: 0.2989
Epoch 106/150
1/1 [==============================] - 0s 262ms/step - loss: 0.1266 - mean_absolute_error: 0.2382 - val_loss: 0.1828 - val_mean_absolute_error: 0.3185
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 146ms/step - loss: 0.1428 - mean_absolute_error: 0.2851
val performance = [0.14283552765846252, 0.28513386845588684]
test performance =  [0.12631037831306458, 0.25823357701301575]


In [87]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 109ms/step - loss: 0.1654 - mean_absolute_error: 0.3028
val performance = [0.1654011458158493, 0.30281156301498413]
test performance =  [0.1397489458322525, 0.2649722993373871]


**performance**

In [88]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
multi_window.compa(elman_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=False)

column : 용존산소
mae:
0.2435686
mse:
0.11289613
rmse
0.3360002
mape
118.50557327270508


In [90]:
#multi_window.compa(elman_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=True)

In [91]:
multi_window.compa(gru_model, plot_col=out_features[0], windows=multi_window.example, min_max_normailze=False)

column : 용존산소
mae:
0.23672602
mse:
0.10526737
rmse
0.32444933
mape
113.5469913482666


In [92]:
# multi_window.compa(gru_model, plot_col=out_features[0], windows=multi_window.example, min_max_normailze=True)

In [93]:
multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=False)

column : 용존산소
mae:
0.20835143
mse:
0.07726374
rmse
0.27796355
mape
83.94021391868591


In [94]:
# multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=True)

In [95]:
multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=False)

column : 용존산소
mae:
0.22089803
mse:
0.08826557
rmse
0.2970952
mape
82.96986222267151


In [96]:
# multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=True)

In [97]:
multi_window.plot24(gru_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …